In [27]:
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib as plt
from bs4 import BeautifulSoup
from requests import get
from time import sleep
from time import time;start_time = time()
from random import randint
from IPython.core.display import clear_output
from warnings import warn


#url = 'https://www.parisattitude.com/fr/louer-appartement/location-meublee/index,locations.aspx?p='
#codesource = requests.get(url).text
#html_soup = BeautifulSoup(codesource, 'html.parser')
#print(html_soup.prettify())
#print(type(html_soup))
#appart_containers = html_soup.find_all('div', class_= 'col-xs-12 col-sm-6 col-md-6 col-lg-4')
#print(appart_containers.prettify())

#csv_file = open('scrapig_parisattitude.csv', 'w')# open a csv file with the option write
#csv_writer = csv.writer(csv_file) 
#csv_writer.writerow(["noms","types_biens","metres_carres","adresses","prix_nuits","prix_mois", "nb_personnes","exclu_nouveaute"])

# Lists to store the scraped data in
noms = []
types_biens = []
nbs_chambres = []
metres_carres = []
adresses = []
quartiers = []
prix_nuits = []
prix_mois = []
nb_personnes = []
#notes_etoiles = []
exclu_nouveaute = []

# Création de listes avec les string correspondant aux nb de pages que l'on souhaite scraper 
pages = [str(i) for i in range(1,4)]

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# On lui énumère les actions qu'il va devoir effectuer pour toutes les pages 
for page in pages:

    # On lui donne l'URL qu'il va devoir scraper en spécifiant les variables sur lesquelles on veut trier notre scraping 
    # Càd qu'on lui précise qu'on veut scraper cet URL lorsque la page = 1 + lorsque la page = 2 (nb de pages précisé L22)
    response = get('https://www.parisattitude.com/fr/louer-appartement/location-meublee/index,locations.aspx?p=' + page)

    # Pause the loop
    sleep(randint(4,20))

    # Monitor the requests
    
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > 72:
        warn('Number of requests was greater than expected.')
        break

    # Parse the content of the request with BeautifulSoup
    html_soup = BeautifulSoup(response.text, 'html.parser')

    # Select all containers from a single page
    appart_containers = html_soup.find_all('div', class_ = 'col-xs-12 col-sm-6 col-md-6 col-lg-4')


    for container in appart_containers:

        nom = container.h2.text
        nom_appart = nom.split()
        noms.append(nom_appart[0])

        types_biens.append(nom_appart[1])

        nbr_de_chambre = [str(s) if s.isdigit() else '0' for s in nom.split(" ",3)[2:-1]]
        nbs_chambres.append(''.join(nbr_de_chambre))

        metre_carree= int(nom_appart[-2])
        metres_carres.append(metre_carree)

        adresse = container.find('span' , class_='adress').text
        adresses.append(adresse)

        code_postal = [str(s) for s in adresse.split() if s.isdigit()]
        quartiers.append(''.join(code_postal))

        prix_par_nuit = float(container.find('span', class_ = 'priceN').text)
        prix_nuits.append(prix_par_nuit)

        prix_par_mois = float(container.find('span', class_ = 'priceM').text)
        prix_mois.append(prix_par_mois)

        nombre_personnes =int(container.find('span', class_ = 'JSMaxPeople').text)
        nb_personnes.append(nombre_personnes)

        exclu_ou_nouveaute = container.find('div', class_ = 'AptNew').text
        exclu_nouveaute.append(exclu_ou_nouveaute.strip())

    # to see whether we’ve scraped our data successfully.    
    liste_appart = pd.DataFrame({'type_contrat': noms,
    'types_biens': types_biens,                            
    'nbs_chambres':nbs_chambres,                            
    'metres_carres': metres_carres,
    'adresses': adresses,
    'quartiers':quartiers,                          
    'prix_nuits': prix_nuits,
    'prix_mois': prix_mois,
    'nb_personnes' : nb_personnes,
    'exclu_nouveaute' : exclu_nouveaute
    })
    print(liste_appart.info())
liste_appart.head(40)

#df_appart_parisattitude = liste_appart
#df_appart_parisattitude.head()
#df_appart_parisattitude.to_excel("appart_attitude_test.xls",index=False)

        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
type_contrat       90 non-null object
types_biens        90 non-null object
nbs_chambres       90 non-null object
metres_carres      90 non-null int64
adresses           90 non-null object
quartiers          90 non-null object
prix_nuits         90 non-null float64
prix_mois          90 non-null float64
nb_personnes       90 non-null int64
exclu_nouveaute    90 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 7.1+ KB
None


,type_contrat,types_biens,nbs_chambres,metres_carres,adresses,quartiers,prix_nuits,prix_mois,nb_personnes,exclu_nouveaute
0,Location,Appartement,0,30,rue Pavee 75004 Paris Le Marais,75004,50.0,1505.0,1,Exclusivité
1,Location,Appartement,1,35,rue des Moines 75017 Paris Batignolles,75017,67.0,2000.0,2,Nouveauté
2,Location,Appartement,0,28,rue Mazarine 75006 Paris Saint Germain Des...,75006,47.0,1398.0,2,Exclusivité
3,Location,Appartement,2,79,rue Maurice Bourdet 75016 Paris Maison de ...,75016,1398.0,3613.0,4,Exclusivité
4,Location,Appartement,0,36,rue de la Faisanderie 75016 Paris rue de l...,75016,50.0,1505.0,2,Exclusivité
5,Location,Appartement,2,49,rue Bernard Palissy 75006 Paris Saint Germ...,75006,99.0,2957.0,4,Exclusivité
6,Location,Appartement,1,65,rue Lucien Micaud 92600 Asnières-sur-Seine ...,92600,54.0,1613.0,2,Exclusivité
7,Location,Appartement,1,35,rue Jouvenet 75016 Paris Exelmans,75016,43.0,1290.0,2,Exclusivité
8,Location,Appartement,3,117,rue Ampere 75017 Paris Wagram,75017,134.0,4032.0,5,Nouveauté
9,Location,Appartement,1,36,rue du Champ de Mars 75007 Paris Champs de...,75007,63.0,1900.0,2,Nouveauté
